In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os

pixel_len=256
color_channel=3
label_size=2

batch_size=50
test_size=100

def input_data(all_dir, tr_fn, te_fn):
    train_file=open(tr_fn,'r')
    test_file=open(te_fn,'r')
    
    lines=train_file.readlines()
    
    X_tr=np.zeros((len(lines),pixel_len,pixel_len,color_channel), np.float16)
    Y_tr=np.zeros((len(lines),label_size))
    
    index=0
    for line in lines:
        filename=line.strip('\n').split(' ')[0]
        label_value=int(line.strip('\n').split(' ')[1])
        img=cv2.imread(all_dir+filename)
        b,g,r=cv2.split(img)
        X_tr[index,:,:,0]=r
        X_tr[index,:,:,1]=g
        X_tr[index,:,:,2]=b
        label_vec=np.zeros(label_size)
        label_vec[label_value]=1
        Y_tr[index]=label_vec
        
        index+=1
        
    lines=test_file.readlines()
    
    X_te=np.zeros((len(lines),pixel_len,pixel_len,color_channel), np.float16)
    Y_te=np.zeros((len(lines),label_size))
    
    index=0
    for line in lines:
        filename=line.strip('\n').split(' ')[0]
        label_value=int(line.strip('\n').split(' ')[1])
        img=cv2.imread(all_dir+filename)
        b,g,r=cv2.split(img)
        X_te[index,:,:,0]=r
        X_te[index,:,:,1]=g
        X_te[index,:,:,2]=b
        label_vec=np.zeros(label_size)
        label_vec[label_value]=1
        Y_te[index]=label_vec
        
        index+=1
        
    return X_tr, Y_tr, X_te, Y_te

def minus_mean(X, mean_image):
    for channel in range(X.shape[3]):
        for elm in range(X.shape[0]):
            X[elm, :, :, channel]-=mean_image[:,:,channel]

def cal_true_class(teY):
    true_class=np.zeros(teY.shape[0])
    for i in range(true_class.shape[0]):
        true_class[i]=np.argmax(teY[i])
    return true_class

In [2]:
main_dir='/home/hr/TensorFlow-Tutorials-master/myPros/'
work_dir=main_dir+'pifu_features_cnn/'
all_dir=main_dir+'256_patch/'

result_i=0

te_fn=work_dir+'x_validate/test'+str(result_i)+'.txt'
tr_fn=work_dir+'x_validate/train'+str(result_i)+'.txt'

print 'load data...'
trX, trY, teX, teY=input_data(all_dir, tr_fn, te_fn)
print 'load data done!'

print 'substract mean image...'
mean_image=np.load('mean_files/mean_'+str(result_i)+'.npy')
minus_mean(trX, mean_image=mean_image)
minus_mean(teX, mean_image=mean_image)
print 'substract mean image done!'

load data...
load data done!
substract mean image...
substract mean image done!


In [3]:
import tensorflow.contrib.slim as slim
import my_inception_v1 as inception_v1

BATCH_SIZE=1

inputs=tf.placeholder(tf.float32, shape=[BATCH_SIZE, pixel_len, pixel_len, color_channel], name='inputs')
labels=tf.placeholder(dtype=tf.int16, shape=[BATCH_SIZE, label_size], name='labels')
p_keep_hidden=tf.placeholder("float")

In [4]:
with slim.arg_scope(inception_v1.vgg_arg_scope()): 
    output, end_points = inception_v1.inception_v1(inputs=inputs, num_classes=label_size,dropout_keep_prob=p_keep_hidden, is_training=False)

prediction = tf.argmax(tf.nn.softmax(output),1)

In [5]:
model_def='models/'+str(result_i)+'/vgg_a_scale_filter.ckpt'
init_fn = slim.assign_from_checkpoint_fn(model_def,
                                        slim.get_model_variables())

In [6]:
train_matrix=np.zeros([label_size,label_size], np.int32)
test_matrix=np.zeros([label_size,label_size], np.int32)

with tf.Session() as sess:
    
    init_fn(sess)

    print 'begin to predict...'
    train_batch = zip(range(0, len(trX), BATCH_SIZE),
                             range(BATCH_SIZE, len(trX)+1, BATCH_SIZE))
    for start, end in train_batch:
        true_class=[np.argmax(v) for v in trY[start:end]]
        pred_class=sess.run(prediction, feed_dict={inputs:trX[start:end], p_keep_hidden:1.0})

        for i,j in zip(true_class, pred_class):
            train_matrix[i][j]+=1
    
    testing_batch = zip(range(0, len(teX), BATCH_SIZE),
                             range(BATCH_SIZE, len(teX)+1, BATCH_SIZE))
    for start, end in testing_batch:
        true_class=[np.argmax(v) for v in teY[start:end]]
        pred_class=sess.run(prediction, feed_dict={inputs:teX[start:end], p_keep_hidden:1.0})

        for i,j in zip(true_class, pred_class):
            test_matrix[i][j]+=1
    print 'predict done!'
    
    train_acc=1.0*np.trace(train_matrix)/np.sum(train_matrix)
    test_acc=1.0*np.trace(test_matrix)/np.sum(test_matrix)

    print(str(result_i)+'th result:')
    print('train_mix_matrix is:')
    print(train_matrix)
    print('train accuracy is: '+str(train_acc))
    print('test_mix_matrix is:')
    print(test_matrix)
    print('test accuracy is: '+str(test_acc))

begin to predict...
predict done!
0th result:
train_mix_matrix is:
[[1350    0]
 [   0 1350]]
train accuracy is: 1.0
test_mix_matrix is:
[[111  39]
 [  0 150]]
test accuracy is: 0.87
